In [1]:
import torch
import torch.nn as nn
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import Dataset,DataLoader
from torchvision import models,datasets,transforms
from torchinfo import summary

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_classes=10
num_epochs=5
batch_size=64
learning_rate=1e-3



In [2]:
data=datasets.CIFAR10(root='./data',train=True,transform=transforms.ToTensor(),download=True)

Files already downloaded and verified


In [3]:
train_loader=DataLoader(dataset=data,batch_size=batch_size,shuffle=True)

In [ ]:
img,label=next(iter(train_loader))

In [9]:
model=torchvision.models.vgg16(pretrained=True)

In [5]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity,self).__init__()
        
    def forward(self,x):
        return x

In [10]:
for param in model.parameters():
        param.requires_grad=False
model.avgpool=Identity()
model.classifier=nn.Linear(512,10)

In [41]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [44]:
summary(model,input_size=img.shape)

Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [64, 10]                  --
├─Sequential: 1-1                        [64, 512, 1, 1]           --
│    └─Conv2d: 2-1                       [64, 64, 32, 32]          (1,792)
│    └─ReLU: 2-2                         [64, 64, 32, 32]          --
│    └─Conv2d: 2-3                       [64, 64, 32, 32]          (36,928)
│    └─ReLU: 2-4                         [64, 64, 32, 32]          --
│    └─MaxPool2d: 2-5                    [64, 64, 16, 16]          --
│    └─Conv2d: 2-6                       [64, 128, 16, 16]         (73,856)
│    └─ReLU: 2-7                         [64, 128, 16, 16]         --
│    └─Conv2d: 2-8                       [64, 128, 16, 16]         (147,584)
│    └─ReLU: 2-9                         [64, 128, 16, 16]         --
│    └─MaxPool2d: 2-10                   [64, 128, 8, 8]           --
│    └─Conv2d: 2-11                      [64, 256, 8, 8]     

In [ ]:
img=img.to(device)
model=model.to(device)
a=model(img)
print(a)

In [7]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [ ]:
#model=model.to(device)
for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(train_loader):
        #labels=labels.type(torch.LongTensor)
        #images=images.view(64,1,28,28)
        #images=images.to(device)
        #labels=labels.to(device)
        preds=model(images)
        loss=criterion(preds,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(loss)

tensor(2.3842, grad_fn=<NllLossBackward0>)


In [32]:
with torch.no_grad():
    pred=model(img)
    print(torch.max(pred,1))
    print(label)

torch.return_types.max(
values=tensor([ 0.6609,  2.8845,  0.4496,  0.6590,  1.9947,  0.8231,  7.6677,  1.8242,
         2.6658,  4.3936,  2.6090,  2.7188,  1.5738,  1.0147,  4.1542,  1.4089,
         1.2017,  2.7490,  0.5536,  4.3098,  4.8601,  2.9283,  2.4579,  1.4203,
         1.7378,  8.0793,  2.4885,  6.6050,  1.5144,  4.2351,  3.9911,  0.8830,
         3.7690,  1.6727,  3.5150, 11.8182,  3.3871,  3.5651,  3.6528,  1.7128,
         1.0004,  1.9216,  2.6288,  2.3533,  1.7929,  1.9538,  5.5470,  1.3782,
         9.2333,  0.4691,  1.7080,  3.1826,  4.4733,  5.3314,  0.6718,  1.4629,
         0.9189,  1.2039,  4.2454,  2.4564,  7.3553,  2.1168,  2.8603,  3.3438],
       device='cuda:0'),
indices=tensor([4, 4, 1, 4, 4, 7, 4, 7, 7, 4, 0, 1, 1, 0, 0, 7, 7, 7, 4, 4, 7, 0, 9, 7,
        7, 4, 9, 0, 4, 7, 4, 7, 4, 0, 4, 7, 7, 7, 7, 4, 4, 4, 4, 0, 4, 4, 4, 4,
        7, 0, 4, 7, 7, 7, 4, 9, 0, 4, 0, 7, 0, 4, 4, 9], device='cuda:0'))
tensor([0, 8, 7, 9, 8, 5, 8, 5, 6, 0, 7, 1, 1, 2, 4, 5, 5, 6